In [10]:
import pandas as pd
from redfin import Redfin
import requests
import os
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=1)

client = Redfin()

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'Addresses_cleaned.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID
0,38204500060410000,RES,2326,LUAU ST,MESQUITE (DALLAS CO),751503826,38204500060410000
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE (DALLAS CO),751491534,38137500370290000
2,60259500060160000,RES,9999,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500060160000
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500050250000
4,60259500060230000,RES,9999,RIPPLEWOOD DR,MESQUITE (DALLAS CO),75181,60259500060230000


In [13]:
# Remove unnesessary (DALLAS CO) from PROPERTY_CITY series
df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()

# Add a column with the full address
df['FullAddress'] = df['STREET_NUM'].astype(str) + ' ' + df['FULL_STREET_NAME'] + ' ' + df['PROPERTY_CITY'] + ' ' + 'TX' + ' ' + df['PROPERTY_ZIPCODE'].astype(str).str[:5]

df = df.drop_duplicates(keep=False, subset=["FullAddress"])

df.head(20)


/var/folders/lm/zm6_v2d953zcr8t5mrvy0_qw0000gn/T/ipykernel_80503/1678436829.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID,FullAddress
0,38204500060410000,RES,2326,LUAU ST,MESQUITE,751503826,38204500060410000,2326 LUAU ST MESQUITE TX 75150
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE,751491534,38137500370290000,1737 HIGHLAND ST MESQUITE TX 75149
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE,75181,60259500050250000,9925 AUTUMN WOOD DR MESQUITE TX 75181
5,38213500050160000,RES,2900,SIDNEY DR,MESQUITE,751502253,38213500050160000,2900 SIDNEY DR MESQUITE TX 75150
9,38139500580020000,RES,1507,RICHARD ST,MESQUITE,751491438,38139500580020000,1507 RICHARD ST MESQUITE TX 75149
10,380441100324A0000,RES,4533,CLUB ESTATES PL,MESQUITE,751501543,380441100324A0000,4533 CLUB ESTATES PL MESQUITE TX 75150
11,500219900E0030000,RES,806,BRANDON DR,SEAGOVILLE,751592398,500219900E0030000,806 BRANDON DR SEAGOVILLE TX 75159
12,65034303040370000,RES,302,S EBRITE ST,MESQUITE,751494635,65034303040370000,302 S EBRITE ST MESQUITE TX 75149
13,38111000020090000,RES,2411,MESQUITE VALLEY RD,MESQUITE,751496647,38111000020090000,2411 MESQUITE VALLEY RD MESQUITE TX 75149
14,500315300F0050000,RES,514,PAYTON CT,SEAGOVILLE,751591783,500315300F0050000,514 PAYTON CT SEAGOVILLE TX 75159


In [17]:
addresses = [df['FullAddress']]

In [ ]:
try:
    client = Redfin()

    address = '2326 LUAU ST MESQUITE TX 75150'

    response = client.search(address)
    url = response['payload']['exactMatch']['url']
    initial_info = client.initial_info(url)

    property_id = initial_info['payload']['propertyId']
    mls_data = client.below_the_fold(property_id)

    # listing_id = initial_info['payload']['listingId']
    avm_details = client.avm_details(property_id, listing_id=None)

    pp.pprint(address)
    print("Response")
    pp.pprint(response)
    print("")

    print("URL")
    pp.pprint(url)
    print("")

    print("Initial Info")
    pp.pprint(initial_info)
    print("")

    print("Propery ID")
    pp.pprint(property_id)
    print("")

    print("MLS Data")
    pp.pprint(mls_data)
    print("")

    # pprint(f'listing id: {listing_id}')

    print("AVM Details")
    pp.pprint(avm_details)
    print("")

except Exception as e:
    pp.pprint(e)

In [ ]:
dict = {}

In [18]:
# create df
house_data_df = pd.DataFrame()

successful = []
unsuccessful = []

for each in addresses[0][0:20]:

    # reset values
    url = ''
    address = each

    try:
        # attempt to get data from each address
        response = client.search(address)
        url = response['payload']['exactMatch']['url']
        initial_info = client.initial_info(url)
        property_id = initial_info['payload']['propertyId']
        mls_data = client.below_the_fold(property_id)

        # if data is successfully found grab info
        baths = mls_data['payload']['publicRecordsInfo']['basicInfo']['baths']
        beds = mls_data['payload']['publicRecordsInfo']['basicInfo']['beds']
        floors = mls_data['payload']['publicRecordsInfo']['basicInfo']['numStories']
        yearBuilt = mls_data['payload']['publicRecordsInfo']['basicInfo']['yearBuilt']
        yearRenovated = mls_data['payload']['publicRecordsInfo']['basicInfo']['yearRenovated']
        sqFtAC = mls_data['payload']['publicRecordsInfo']['basicInfo']['sqFtFinished']
        sqFtInterior = mls_data['payload']['publicRecordsInfo']['basicInfo']['totalSqFt']
        sqFtLot = mls_data['payload']['publicRecordsInfo']['basicInfo']['lotSqFt']

        latlong = initial_info['payload']['latLong']
        img = initial_info['payload']['preloadImageUrls']

        # add to house_data_df
        houseData = {
            'property_id': property_id,
            'address': address,
            'beds': beds,
            'baths': baths,
            'floors': floors,
            'yearBuilt': yearBuilt,
            'yearRenovated': yearRenovated,
            'sqFtAC': sqFtAC,
            'sqFtInterior': sqFtInterior,
            'sqFtLot': sqFtLot,
            'imgURL': img
        }

        # append data to df here
        successful.append(address)


    except Exception as e:

        # anything that doesn't have above info gets thrown as error
        unsuccessful.append(address)
        print(f'error: {e} at address: {address}')

error: 'baths' at address: 9925 AUTUMN WOOD DR MESQUITE TX 75181
error: 'exactMatch' at address: 4533 CLUB ESTATES PL MESQUITE TX 75150
error: 'baths' at address: 1202 BOWERS RD SEAGOVILLE TX 75159
error: 'exactMatch' at address: 2407 E SEAGOVILLE RD SEAGOVILLE TX 75159
error: 'baths' at address: 922 AUTUMN WOOD DR MESQUITE TX 75181
error: 'baths' at address: 9924 AUTUMN WOOD DR MESQUITE TX 75181
error: 'exactMatch' at address: 2101 SILVER LEAF DR MESQUITE TX 75181


In [ ]:
# client = Redfin()

# successful = []
# unsuccessful = []

# for index, each in enumerate(addresses[0][0:10]):
    
#     address = each
#     url=''
#     temp_list = []
#     try:
#         response = client.search(address)
#         url = response['payload']['exactMatch']['url']

#         if "url" in dict:
#         # Append the value to the existing list
#             dict["url"].append(url)
#         else:
#             # Create a new key-value pair with a list containing the new value
#             dict["url"] = [url]

#     except Exception as e:
#         unsuccessful.append((index, address))
#         # print("*!*!*!*!*!*!*!*!*!")
#         # print(e)
#         # print(address)
#         # print("*^*^*^^*^*^*")
#     # initial_info = client.initial_info(url)

#     # property_id = initial_info['payload']['propertyId']
#     # mls_data = client.below_the_fold(property_id)

#     # # listing_id = initial_info['payload']['listingId']
#     # avm_details = client.avm_details(property_id)
#     # pp.pprint(url)

# pp.pprint(dict)

In [ ]:
pp.pprint(dict)

In [ ]:
total = len(unsuccessful) + len(successful)
percentage = len(unsuccessful)/total * 100
# print(percentage)
# print(total)

In [ ]:
print(len(successful))
print(len(unsuccessful))